<a href="https://colab.research.google.com/github/telecombcn-dl/2018-dlai-team10/blob/master/MLP_DLAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

In [0]:
import numpy as np
import os
import torch
import torchvision
import codecs
import torch.utils.data
from torchvision import datasets, transforms
import torchvision.datasets as datasets
import torch.nn as nn
from matplotlib import pyplot as plt
from PIL import Image
from tqdm import tqdm


In [0]:
  urls = [
        'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/key.npy',
        'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/banana.npy',
        'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/ladder.npy',
        'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/tennis%20racquet.npy',
        'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/pizza.npy',
        'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/stop%20sign.npy',
        'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/wheel.npy',
        'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/fork.npy',
        'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/book.npy',
        'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/apple.npy',
    ]
    
  def download_url(self, url, root, filename):
      from six.moves import urllib
      root = os.path.expanduser(root)
      fpath = os.path.join(root, filename)

      self.createDir(root)

      # downloads file
      if os.path.isfile(fpath):
          a = 1
          #print('Using downloaded and verified file: ' + fpath)
      else:
          try:
              print('Downloading ' + url + ' to ' + fpath)
              urllib.request.urlretrieve(
                  url, fpath,
                  reporthook=self.gen_bar_updater(tqdm(unit='B', unit_scale=True))
              )
          except OSError:
              if url[:5] == 'https':
                  url = url.replace('https:', 'http:')
                  print('Failed download. Trying https -> http instead.'
                        ' Downloading ' + url + ' to ' + fpath)
                  urllib.request.urlretrieve(
                      url, fpath,
                      reporthook=self.gen_bar_updater(tqdm(unit='B', unit_scale=True))
                  )
    
    

In [0]:
"""
This script takes the raw dataset from a given directory, divides and reshapes it into single images
and places those images in their also created by the script corresponding folders.
The folder structure is the following:
Inside 'dire', there are npy files for each class. This script creates three folders: train, validation and test. 
Inside each of these three folders, the script creates one folder for each class and places there the corresponding
reshaped and divided dateset files. 
"""

class_name = ['apple', 'banana', 'book', 'fork', 'key', 'ladder', 'pizza', 'stop_sign', 'tennis_racquet', 'wheel']
step = ['train', 'validation', 'test']

dire = r'data/DrawDataset/raw/'

max_length = 100000 # Maximum number of files (drawings) per class
percen=[0.6, 0.3, 0.1] # Percentage of training, validation and testing

begin = [0, int(max_length * percen[0]), int(max_length * (percen[0] + percen[1])) + 1]
end = [int(max_length * (percen[0])), int(max_length * (percen[0] + percen[1])) + 1, max_length]

#for c in range(0, len(class_name)):
for c in range(0, 3):
	filename = dire + str(class_name[c]) + '.npy'
	data = np.load(filename)

	for s in range(0, len(step)):
		dire_step = str(dire) + str(step[s])
		if not os.path.exists(dire_step):
			os.makedirs(dire_step)

		for i in range(begin[s], end[s]):
			dire_class = str(dire_step) + '/' + str(class_name[c])
			if not os.path.exists(dire_class):
				os.makedirs(dire_class)
        
			# Reshape the raw data into 28x28 images
			data_sample = data[i,:].reshape((28, 28))
			sample_name = class_name[c] + '_' + str(step[s]) + '_' + str(i)
			np.save(os.path.join(dire_class, sample_name), data_sample)

In [35]:
!ls data/DrawDataset/raw
!ls data/DrawDataset/raw/test
!ls data/DrawDataset/raw/train
!ls data/DrawDataset/raw/validation
!ls data/DrawDataset/raw/test/apple

apple.npy   fork.npy	pizza.npy	    test	wheel.npy
banana.npy  key.npy	stop_sign.npy	    train
book.npy    ladder.npy	tennis_racquet.npy  validation
apple  banana  book  fork  key	ladder	pizza  stop_sign  tennis_racquet  wheel
apple  banana  book  fork  key	ladder	pizza  stop_sign  tennis_racquet  wheel
apple  banana  book  fork  key	ladder	pizza  stop_sign  tennis_racquet  wheel
apple_test_90000.npy  apple_test_93334.npy  apple_test_96668.npy
apple_test_90001.npy  apple_test_93335.npy  apple_test_96669.npy
apple_test_90002.npy  apple_test_93336.npy  apple_test_96670.npy
apple_test_90003.npy  apple_test_93337.npy  apple_test_96671.npy
apple_test_90004.npy  apple_test_93338.npy  apple_test_96672.npy
apple_test_90005.npy  apple_test_93339.npy  apple_test_96673.npy
apple_test_90006.npy  apple_test_93340.npy  apple_test_96674.npy
apple_test_90007.npy  apple_test_93341.npy  apple_test_96675.npy
apple_test_90008.npy  apple_test_93342.npy  apple_test_96676.npy
apple_test_90009.npy  apple_test_

In [37]:
def load_sample(x):
	return np.load(x)

train_dir = r"data/DrawDataset/raw/train"
val_dir = r"data/DrawDataset/raw/validation"
test_dir = r"data/DrawDataset/raw/test"

train_dataset = datasets.DatasetFolder(train_dir, extensions = ['.npy'], loader = load_sample)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 4, shuffle = True, num_workers = 0)
data_iter = iter(train_loader)

# El que es printa hauria de ser un batch que entra per fer el forward pass i el seu backward pass corresponent quan entrenem
batch, labels = data_iter.next()
print(batch.size())

torch.Size([4, 28, 28])
